[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/03-prompts.ipynb)

### LangChain Essentials

# Prompts Templating for OpenAI - LangChain

Until 2021, to use an AI model for a specific use-case we would need to fine-tune the model weights themselves. That would require huge amounts of training data and significant compute to fine-tune any reasonably performing model.

Instruction fine-tuned **L**arge **L**anguage **M**odels (LLMs) changed this fundamental rule of applying AI models to new use-cases. Rather than needing to either train a model from scratch or fine-tune an existing model, these new LLMs could adapt incredibly well to a new problem or use-case with nothing more than a prompt change.

Prompts allow us to completely change the functionality of an AI pipeline. Through natural language we simply _tell_ our LLM what it needs to do, and with the right AI pipeline and prompting, it often works.

LangChain naturally has many functionalities geared towards helping us build our prompts. We can build very dynamic prompting pipelines that modifying the structure and content of what we feed into our LLM based on essentially any parameter we would like. In this example, we'll explore the essentials to prompting in LangChain and apply this in a demo **R**etrieval **A**ugmented **G**eneration (RAG) pipeline.


In [1]:
!pip uninstall -y langchain google-generativeai
!pip install -qU \
  "pydantic>=2.11.4,<3" \
  "langchain-core>=0.3.70,<0.4" \
  "langchain-community>=0.3.27,<0.4" \
  "langchain-google-genai>=2.1.10" \
  "google-genai>=0.7.0" \
  "langsmith>=0.3.4"


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.7/241.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

> Switched to Gemini



---

> ⚠️ If using LangSmith, add your API key below:

In [3]:
import os
from getpass import getpass
from google.colab import userdata

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-prompts-openai"

---

## Basic Prompting

We'll start by looking at the various parts of our prompt. For RAG use-cases we'll typically have three core components however this is _very_ use-cases dependant and can vary significantly. Nonetheless, for RAG we will typically see:

* **Rules for our LLM**: this part of the prompt sets up the behavior of our LLM, how it should approach responding to user queries, and simply providing as much information as possible about what we're wanting to do as possible. We typically place this within the _system prompt_ of an chat LLM.

* **Context**: this part is RAG-specific. The context refers to some _external information_ that we may have retrieved from a web search, database query, or often a _vector database_. This external information is the **R**etrieval **A**ugmentation part of **RA**G. For chat LLMs we'll typically place this inside the chat messages between the assistant and user.

* **Question**: this is the input from our user. In the vast majority of cases the question/query/user input will always be provided to the LLM (and typically through a _user message_). However, the format and location of this being provided often changes.

* **Answer**: this is the answer from our assistant, again this is _very_ typical and we'd expect this with every use-case.

The below is an example of how a RAG prompt may look:

```
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
```

Here we can see how the AI will appoach our question, as you can see we have a formulated response, if the context has the answer, then use the context to answer the question, if not, say I don't know, then we also have context and question which are being passed into this similarly to paramaters in a function.

In [4]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

LangChain uses a `ChatPromptTemplate` object to format the various prompt types into a single list which will be passed to our LLM:

In [5]:
from langchain.prompts import ChatPromptTemplate

# passing the template to the LangChain model
prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

When we call the template it will expect us to provide two variables, the `context` and the `query`. Both of these variables are pulled from the strings we wrote, as LangChain interprets curly-bracket syntax (ie `{context}` and `{query}`) as indicating a dynamic variable that we expect to be inserted at query time. We can see that these variables have been picked up by our template object by viewing it's `input_variables` attribute:

In [6]:
prompt_template.input_variables

['context', 'query']

We can also view the structure of the messages (currently _prompt templates_) that the `ChatPromptTemplate` will construct by viewing the `messages` attribute:

In [7]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

From this, we can see that each tuple provided when using `ChatPromptTemplate.from_messages` becomes an individual prompt template itself. Within each of these tuples, the first value defines the _role_ of the message, which is typically `system`, `human`, or `ai`. Using these tuples is shorthand for the following, more explicit code:

In [8]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

We can see the structure of this new chat prompt template is identical to our previous:

In [9]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

### Invoking our LLM with Templates

We start by initializing our LLM. We will use Gemini's `Flash 2.5 Lite` model, if you need an API key you can get one from Google AI studio.



In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY") or getpass(
    "Enter Gemini API Key: "
)

openai_model = "gemini-2.5-flash-lite"

llm = ChatGoogleGenerativeAI(
    model=openai_model,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

Here we define our LLM and _because_ we're using it for a question-answer use-case we want it's answer to be as grounded in reality as possible. To do that, we ofcourse prompt it to not make up any information via the `If you cannot answer the question using the provided information answer with "I don't know"` line, but we _also_ use the model's `temperature` setting.

The `temperature` parameter controls the randomness of the LLM's output. A temperature of `0.0` makes an LLM's output more determinstic which _in theory_ should lead to a lower likelihood of hallucination.

Now, the question here may be, _why would we ever not use `temperature=0.0`?_ The answer to that is that sometimes a little bit of randomness can useful. Randomness tends to translate to text that feels more human and creative, so if we'd like an LLM to help us write an article or even a poem, that lack of determinism becomes a feature rather than a bug.

For now, we'll stick with our more deterministic LLM. We'll setup the pipeline to consume two variables when our LLM pipeline is called, `query` and `context`, we'll feed them into our chat prompt template, and then invoke our LLM with our formatted messages.

Although that sounds complicated, all we're doing is connecting our `prompt_template` and `llm`. We do this with **L**ang**C**hain **E**xpression **L**anguage (LCEL), which uses the `|` operator to connect our each component.

In [11]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template
    | llm
)

Now let's define a `query` and some relevant `context` and invoke our pipeline.

In [12]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [13]:
pipeline.invoke({"query": query, "context": context})

AIMessage(content='Aurelio AI is an AI company that develops tooling for AI engineers, with a particular focus on language AI. They have expertise in building AI agents and information retrieval.\n\nThe company offers:\n*   **Open-source frameworks:** Such as Semantic Router and Semantic Chunkers.\n*   **An AI Platform:** Providing tools for AI engineers.\n*   **Development services:** Helping other organizations bring their AI technology to market.\n\nAurelio AI also became LangChain Experts in September 2024, indicating a strong history of delivering solutions using the LangChain ecosystem.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--5502d40d-34cb-4213-a532-f518c16e8c48-0', usage_metadata={'input_tokens': 183, 'output_tokens': 119, 'total_tokens': 302, 'input_token_details': {'cache_read': 0}})

Our LLM pipeline is able to consume the information from the `context` and use it to answer the user's `query`. Ofcourse, we would not usually be feeding in both a question and an answer into an LLM manually. Typically, the `context` would be retrieved from a vector database, via web search, or from elsewhere. We will cover this use-case in full and build a functional RAG pipeline in a future chapter.

## Few Shot Prompting

Many **S**tate-**o**f-**t**he-**A**rt (SotA) LLMs are incredible at instruction following. Meaning that it requires much less effort to get the intended output or behavior from these models than is the case for older LLMs and smaller LLMs.

Before creating an example let's first see how to use LangChain's few shot prompting objects. We will provide multiple examples and we'll feed them in as sequential human and ai messages so we setup the template like this:

In [14]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

Then we define a list of examples with dictionaries containing the correct `input` and `output` keys.

In [15]:
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

We then feed both of these into our `FewShotChatMessagePromptTemplate` object:

In [16]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# here is the formatted prompt
print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


Using this we can provide different sets of `examples` or even different individual `example_prompt` templates to the `FewShotChatMessagePromptTemplate` object to build our prompt structure. Let's try an real example where we might use few-shot prompting.

### Few-Shot Example

Using a tiny LLM limits it's ability, so when asking for specific behaviors or structured outputs it can struggle. For example, we'll ask the LLM to summarize the key points about Aurelio AI using markdown and bullet points. Let's see what happens.

In [17]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context}).content
print(out)

# Aurelio AI: AI Engineering Tooling and Services

Aurelio AI is a company dedicated to developing tools and providing services for AI engineers, with a particular emphasis on language AI.

## Key Offerings:

*   **AI Tooling:**
    *   Development of open-source frameworks such as Semantic Router and Semantic Chunkers.
    *   An AI Platform that offers engineers tools for building AI applications.
*   **Expertise:**
    *   Strong focus on language AI.
    *   Deep expertise in building AI agents.
    *   Significant background in information retrieval.
*   **Services:**
    *   Development services for organizations looking to commercialize their AI technology.
*   **LangChain Partnership:**
    *   Became LangChain Experts in September 2024, demonstrating a strong history of building solutions using the LangChain ecosystem.

In summary, Aurelio AI empowers AI engineers through specialized tooling, open-source contributions, and expert development services, particularly within the r

We can display our markdown nicely with `IPython` like so:

In [18]:
from IPython.display import display, Markdown

display(Markdown(out))

# Aurelio AI: AI Engineering Tooling and Services

Aurelio AI is a company dedicated to developing tools and providing services for AI engineers, with a particular emphasis on language AI.

## Key Offerings:

*   **AI Tooling:**
    *   Development of open-source frameworks such as Semantic Router and Semantic Chunkers.
    *   An AI Platform that offers engineers tools for building AI applications.
*   **Expertise:**
    *   Strong focus on language AI.
    *   Deep expertise in building AI agents.
    *   Significant background in information retrieval.
*   **Services:**
    *   Development services for organizations looking to commercialize their AI technology.
*   **LangChain Partnership:**
    *   Became LangChain Experts in September 2024, demonstrating a strong history of building solutions using the LangChain ecosystem.

In summary, Aurelio AI empowers AI engineers through specialized tooling, open-source contributions, and expert development services, particularly within the realm of language AI and the LangChain framework.

This is not bad, but also not quite the format we wanted. We could try improving our initial prompting instructions, but when this doesn't work we can move on to our few-shot prompting. We want to build something like this:

```
Answer the user's query based on the context below,                 }
if you cannot answer the question using the                         }
provided information answer with "I don't know"                     }
                                                                    }--->  (Rules)
Always answer in markdown format. When doing so please              }
provide headers, short summaries, follow with bullet                }
points, then conclude. Here are some examples:                      }


User: Can you explain gravity?                                      }
AI: ## Gravity                                                      }
                                                                    }
Gravity is one of the fundamental forces in the universe.           }
                                                                    }
### Discovery                                                       }--->  (Example 1)
                                                                    }
* Gravity was first discovered by...                                }
                                                                    }
**To conclude**, Gravity is a fascinating topic and has been...     }
                                                                    }

User: What is the capital of France?                                }
AI: ## France                                                       }
                                                                    }
The capital of France is Paris.                                     }
                                                                    }--->  (Example 2)
### Origins                                                         }
                                                                    }
* The name Paris comes from the...                                  }
                                                                    }
**To conclude**, Paris is highly regarded as one of the...          }

Context: {context}                                                  }--->  (Context)
```

We have already defined our `example_prompt` so now we just change our `examples` to use some examples of a user asking a question and the LLM answering in the exact markdown format we need.

In [19]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

We feed these into our `FewShotChatMessagePromptTemplate` object:

In [20]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

Our formatted prompt now looks like this:

In [21]:
out = few_shot_prompt.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



We then pull all of this together with our system prompt and final user query to create our final prompt and feed it into our LLM.

In [22]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'Can you explain gravity?', 'output': '## Gravity\n\nGravity is one of the fundamental forces in the universe.\n\n### Discovery\n\n* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n### In General Relativity\n\n* Gravity is described as the curvature of spacetime.\n* The more massive an object is, the more it curves spacetime.\n* This curvature is what causes objects to fall towards each other.\n\n### Gravitons\n\n* Gravitons are hypothetical particles that mediate the force of gravity.\n* They have not yet been detected.\n\n**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n'}, {'input': 'What is the capital of France?', 'output': '## France\n\nThe capital of France is Paris.\n\n### Origins\n\n* The name Paris comes from the Latin word "Parisini" which refe

In [23]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

Now feed this back into our pipeline:

In [24]:
pipeline = prompt_template | llm
out = pipeline.invoke({"query": query, "context": context}).content
display(Markdown(out))

## Aurelio AI's Offerings

Aurelio AI is a company that develops tooling specifically for AI engineers.

### Core Focus

*   **Language AI:** Their primary area of expertise lies in language artificial intelligence.
*   **AI Agents & Information Retrieval:** The team possesses strong skills in building AI agents and has a solid background in information retrieval.

### Products and Services

*   **Open Source Frameworks:** Aurelio AI is the creator of notable open-source frameworks such as:
    *   Semantic Router
    *   Semantic Chunkers
*   **AI Platform:** They offer an AI Platform designed to provide engineers with the necessary tools to build AI applications.
*   **Development Services:** The company also provides development services to other organizations, assisting them in bringing their AI technologies to market.
*   **LangChain Experts:** Since September 2024, Aurelio AI has been recognized as LangChain Experts, a designation earned through their consistent delivery of AI solutions utilizing the LangChain ecosystem.

**In conclusion**, Aurelio AI is a specialized company supporting AI engineers through open-source tools, a dedicated platform, and expert development services, with a particular emphasis on language AI and the LangChain framework.

We can see that by adding a few examples to our prompt, ie _few-shot prompting_, we can get much more control over the exact structure of our LLM response. As the size of our LLMs increases, the ability of them to follow instructions becomes much greater and they tend to require less explicit prompting as we have shown here. However, even for SotA models like `gpt-4o` few-shot prompting is still a valid technique that can be used if the LLM is struggling to follow our intended instructions.

## Chain of Thought Prompting

We'll take a look at one more commonly used prompting technique called _chain of thought_ (CoT). CoT is a technique that encourages the LLM to think through the problem step by step before providing an answer. The idea being that by breaking down the problem into smaller steps, the LLM is more likely to arrive at the correct answer and we are less likely to see hallucinations.

To implement CoT we don't need any specific LangChain objects, instead we are simply modifying how we instruct our LLM within the system prompt. We will ask the LLM to list the problems that need to be solved, to solve each problem individually, and then to arrive at the final answer.

Let's first test our LLM _without_ CoT prompting.

In [28]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation. Dont break down problem into steps
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}"),
])

Nowadays most LLMs are trained to use CoT prompting by default, so we actually need to instruct it not to do so for this example which is why we added `"You MUST answer the question directly without any other text or explanation."` to our system prompt.

In [29]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

no_cot_pipeline = no_cot_prompt_template | llm
no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

1392


The actual answer is `1392`, but the LLM _without_ CoT just hallucinates and gives us a guess. Now, we can add explicit CoT prompting to our system prompt to see if we can get a better result.

In [30]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm

In [31]:
cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

Here's a breakdown of the keystrokes needed to type the numbers from 1 to 500:

**Subproblems:**

1.  **Count the number of single-digit numbers:** Identify how many numbers from 1 to 500 are single-digit.
2.  **Count the number of two-digit numbers:** Identify how many numbers from 1 to 500 are two-digit.
3.  **Count the number of three-digit numbers:** Identify how many numbers from 1 to 500 are three-digit.
4.  **Calculate keystrokes for single-digit numbers:** Determine the total keystrokes for all single-digit numbers.
5.  **Calculate keystrokes for two-digit numbers:** Determine the total keystrokes for all two-digit numbers.
6.  **Calculate keystrokes for three-digit numbers:** Determine the total keystrokes for all three-digit numbers.
7.  **Sum the keystrokes:** Add the keystrokes from all categories to get the final answer.

**Solving the Subproblems:**

1.  **Count the number of single-digit numbers:**
    *   The single-digit numbers are 1, 2, 3, 4, 5, 6, 7, 8, 9.
    *   There are **9** single-digit numbers.

2.  **Count the number of two-digit numbers:**
    *   The two-digit numbers range from 10 to 99.
    *   To count them, we can use the formula: Last Number - First Number + 1.
    *   So, 99 - 10 + 1 = **90** two-digit numbers.

3.  **Count the number of three-digit numbers:**
    *   The three-digit numbers range from 100 to 500.
    *   Using the same formula: 500 - 100 + 1 = **401** three-digit numbers.

4.  **Calculate keystrokes for single-digit numbers:**
    *   Each single-digit number requires 1 keystroke.
    *   Total keystrokes = 9 numbers * 1 keystroke/number = **9** keystrokes.

5.  **Calculate keystrokes for two-digit numbers:**
    *   Each two-digit number requires 2 keystrokes.
    *   Total keystrokes = 90 numbers * 2 keystrokes/number = **180** keystrokes.

6.  **Calculate keystrokes for three-digit numbers:**
    *   Each three-digit number requires 3 keystrokes.
    *   Total keystrokes = 401 numbers * 3 keystrokes/number = **1203** keystrokes.

7.  **Sum the keystrokes:**
    *   Total keystrokes = Keystrokes (1-digit) + Keystrokes (2-digit) + Keystrokes (3-digit)
    *   Total keystrokes = 9 + 180 + 1203 = **1392** keystrokes.

**Final Answer:**

To type the numbers from 1 to 500, a total of **1392** keystrokes are needed.

Now we get a much better result! Our LLM provides us with a final answer of `1392` which is correct. Finally, as mentioned most LLMs are now trained to use CoT prompting by default. So let's see what happens if we don't explicitly tell the LLM to use CoT.

In [32]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

pipeline = prompt_template | llm

In [33]:
result = pipeline.invoke({"query": query}).content
display(Markdown(result))

Here's how to calculate the number of keystrokes needed to type the numbers from 1 to 500:

**1. Single-Digit Numbers (1-9):**
* There are 9 single-digit numbers.
* Each requires 1 keystroke.
* Total keystrokes: 9 * 1 = 9

**2. Double-Digit Numbers (10-99):**
* There are 99 - 10 + 1 = 90 double-digit numbers.
* Each requires 2 keystrokes.
* Total keystrokes: 90 * 2 = 180

**3. Triple-Digit Numbers (100-500):**
* There are 500 - 100 + 1 = 401 triple-digit numbers.
* Each requires 3 keystrokes.
* Total keystrokes: 401 * 3 = 1203

**4. Total Keystrokes:**
* Add the keystrokes from each category: 9 + 180 + 1203 = **1392**

Therefore, you need **1392** keystrokes to type the numbers from 1 to 500.

We almost get the _exact_ same result. The formatting isn't quite as nice but the CoT behavior is clearly there, and the LLM produces the correct final answer!

CoT is useful not only for simple question-answering like this, but is also a fundamental component of many agentic systems which will often use CoT steps paired with tool use to solve very complex problems, this is what we see in OpenAI's current flagship model `o1`. We'll see later in the course how we can do this ourselves.